In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time

# Initialize the URL for the Premier League stats page
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

# Fetch the page data
data = requests.get(standings_url)

# Parse the page content using BeautifulSoup
soup = BeautifulSoup(data.text)
tables = soup.select('table.stats_table')
if tables:
    standings_table = tables[0]
else:
    print("No table found.")

# Extract all links from the standings table
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]
team_urls = [f"https://fbref.com{l}" for l in links]

# Fetch data for the first team
data = requests.get(team_urls[0])

# Read match data from the page
matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
soup = BeautifulSoup(data.text)

# Extract links for shooting data
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]

# Fetch shooting data
data = requests.get(f"https://fbref.com{links[0]}")
shooting = pd.read_html(data.text, match="Shooting")[0]
shooting.columns = shooting.columns.droplevel()

# Merge match data with shooting data
team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
team_data.head()

# Define the years for which data is required
years = list(range(2022, 2020, -1))
all_matches = []

# Loop through each year and fetch data
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    # Get previous season's link for the next iteration
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        
        # Fetch shooting data for the team
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Premier League"]
        
        # Add season and team name columns
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        
        # Add delay to avoid being blocked by the server
        time.sleep(1)

# Concatenate all team match data
match_df = pd.concat(all_matches)

match_df.columns = [c.lower() for c in match_df.columns]

# Save the final DataFrame to a CSV file
match_df.to_csv("matches.csv", index=False)
